# DB2-Salesforce connector: Tools mapping to users

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')


salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors


Obtained Salesforce access token ...... True


In [2]:
import pandas as pd
import time
import datetime

## Obtain tool information from DB2 

In [3]:
# connect with DB2

# toolstart
sql_query = "select user, tool as instance from nanohub_metrics.toolstart"

toolstart_df = pd.read_sql_query(sql_query, nanohub_metrics_db)

# tool_version
sql_query = "select toolname, instance from nanohub.jos_tool_version"

tool_version_df = pd.read_sql_query(sql_query, nanohub_db)

In [4]:
# convert all to lower case
toolstart_df['user'] = toolstart_df['user'].apply(str.lower)
toolstart_df['instance'] = toolstart_df['instance'].apply(str.lower)

tool_version_df['toolname'] = tool_version_df['toolname'].apply(str.lower)
tool_version_df['instance'] = tool_version_df['instance'].apply(str.lower)

# display

display(toolstart_df.head(2))
display(tool_version_df.head(2))

,user,instance
0,herre,cnt-iv
1,donovan,huckel-iv


,toolname,instance
0,,
1,112016a,112016a_dev


In [5]:
# insert the toolname into instance as well
dummy = pd.DataFrame()
dummy['toolname'] = tool_version_df['toolname'].drop_duplicates()
dummy['instance'] = dummy['toolname']

tool_version_df = pd.concat([dummy, tool_version_df], axis=0).drop_duplicates().reset_index(drop=True)

# display
tool_version_df.head(3)

,toolname,instance
0,,
1,112016a,112016a
2,1dbtetransient,1dbtetransient


In [6]:
# merge toolname into toolstart
df = pd.merge(toolstart_df, tool_version_df, how='left', left_on='instance', right_on='instance')[['user', 'toolname']]

In [7]:
# count number of tool runs by each user, each tool
df = df.groupby(['user', 'toolname'])['toolname'].count().to_frame(name='cnt').reset_index()

In [8]:
# just in case some NaN values made through
df = df[~df.user.isna()]
df = df[~df.toolname.isna()]

# display
df.head(3)

,user,toolname,cnt
0,,,885
1,0.yao.yuan,mosfet,11
2,0.yao.yuan,mosfetsat,4


## Obtain Salesforce IDs

In [9]:
# create DB2 to Salesforce API object

In [10]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_username_df = db_s.query_data('SELECT Id, nanoHUB_username__c FROM Contact where nanoHUB_username__c != NULL')

# display
sf_username_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000b2DBOAA2


{"id":"7505w00000b2DBOAA2","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:45:07.000+0000","systemModstamp":"2021-10-25T11:45:07.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000b2DBOAA2","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:45:07.000+0000","systemModstamp":"2021-10-25T11:45:18.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":262426,"retries":0,"totalProcessingTime":9620}
[Success] Bulk job completed successfully.


,Id,nanoHUB_username__c
0,0035w000031Vsp1AAC,hubrepo
1,0035w000031Vsp2AAC,support
2,0035w000031Vsp3AAC,gridstat


In [11]:
# get Salesforce ID for citations
sf_tool_df = db_s.query_data('SELECT Id, Tool_name__c FROM nanoHUB_tools__c')

# display
sf_tool_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000b2DBYAA2


{"id":"7505w00000b2DBYAA2","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:45:22.000+0000","systemModstamp":"2021-10-25T11:45:22.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000b2DBYAA2","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-25T11:45:22.000+0000","systemModstamp":"2021-10-25T11:45:24.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":1823,"retries":0,"totalProcessingTime":234}
[Success] Bulk job completed successfully.


,Id,Tool_name__c
0,a0s5w00000k5MOwAAM,ellipsom
1,a0s5w00000k5MOxAAM,hydrolab
2,a0s5w00000k5MOyAAM,huckel


## Match data with Salesforce format

In [12]:
# make sure all strings are lower cases

sf_username_df['nanoHUB_username__c'] = sf_username_df['nanoHUB_username__c'].astype('str').apply(str.lower)

sf_tool_df['Tool_name__c'] = sf_tool_df['Tool_name__c'].astype('str').apply(str.lower)

In [13]:
# merge SF citation and contact IDs into user-citation DF
ct_tolink_df = pd.merge(df, sf_tool_df, how='inner', left_on='toolname', right_on='Tool_name__c')\
                           .rename(columns={'Id':'SF_ID_tool'})
                                            
ct_tolink_df = pd.merge(ct_tolink_df, sf_username_df, how='inner', left_on='user', right_on='nanoHUB_username__c')\
                           .rename(columns={'Id':'SF_ID_contact'})

# display
ct_tolink_df.head(2)

,user,toolname,cnt,SF_ID_tool,Tool_name__c,SF_ID_contact,nanoHUB_username__c
0,0.yao.yuan,mosfet,11,a0s5w00000k5MPbAAM,mosfet,0035w000034IX15AAG,0.yao.yuan
1,0.yao.yuan,mosfetsat,4,a0s5w00000k5Mc5AAE,mosfetsat,0035w000034IX15AAG,0.yao.yuan


In [14]:
# create a new column for object name
ct_tolink_df['Name'] = ct_tolink_df.apply(lambda x: '%s_%s'%(x.nanoHUB_username__c, x.Tool_name__c), axis=1)

In [15]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name']         = ct_tolink_df['Name']
df_sf['Contact__c'] = ct_tolink_df['SF_ID_contact']
df_sf['Tool__c']       = ct_tolink_df['SF_ID_tool']
df_sf['Total_Runs__c'] = ct_tolink_df['cnt']
sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,0.yao.yuan_mosfet,0.yao.yuan_mosfetsat
Contact__c,0035w000034IX15AAG,0035w000034IX15AAG
Tool__c,a0s5w00000k5MPbAAM,a0s5w00000k5Mc5AAE
Total_Runs__c,11,4


## To Salesforce Sales Cloud CRM

In [16]:
# create DB2 to Salesforce API object

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

In [17]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000b2DBdAAM
hello


[Success] CSV upload successful. Job ID = 7505w00000b2DBdAAM


[Success] Closing job successful. Job ID = 7505w00000b2DBdAAM


In [18]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-10-25T11:45:42.000+0000',
 'externalIdFieldName': 'Name',
 'id': '7505w00000b2DBdAAM',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'ContactToolAssociation__c',
 'operation': 'upsert',
 'retries': 0,
 'state': 'UploadComplete',
 'systemModstamp': '2021-10-25T11:46:20.000+0000',
 'totalProcessingTime': 0}
